# Задача классификации изображений

In [29]:
import torchvision.transforms as transforms
import torch
from torchvision import datasets
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [30]:
transform = transforms.Compose([
    transforms.Resize(32,),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

train_size = int(len(dataset) * 0.6)
val_size = int(len(dataset) * 0.2)
test_size = len(dataset) - (val_size + train_size)

generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

In [31]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, padding=1, stride=1)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, padding=1, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 16 * 16, 32)
        self.fc2 = nn.Linear(32, 10)

        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [33]:
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

patience = 2
best_val_loss = float("inf")
patience_counter = 0
model_path = "best_cnn_classification.pth"

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_train = 0
  total_train = 0

  for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_train += (preds == labels).sum().item()
    total_train += labels.size(0)

  train_loss = running_loss /  len(train_loader)
  train_accuracy = correct_train / total_train

  model.eval()
  val_loss = 0.0
  correct_val = 0
  total_val = 0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      preds = torch.argmax(outputs, dim=1)
      correct_val += (preds == labels).sum().item()
      total_val += labels.size(0)

  val_loss /= len(val_loader)
  val_accuracy = correct_val / total_val


  print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    patience_counter = 0
    torch.save(model.state_dict(), model_path)
    print(f"✔ Model improved. Saved to {model_path}")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print("Ранняя остановка сработала!")
      break

model.load_state_dict(torch.load(model_path))
model.to(device)
print(f"✅ Best model loaded from {model_path}")

model.eval()
test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    test_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_test += (preds == labels).sum().item()
    total_test += labels.size(0)

test_loss /= len(test_loader)
test_accuracy = correct_test / total_test
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5: 100%|██████████| 469/469 [06:38<00:00,  1.18it/s]


Epoch 1, Train Loss: 1.8712, Val Loss: 1.4935
✔ Model improved. Saved to best_cnn_classification.pth


Epoch 2/5: 100%|██████████| 469/469 [06:33<00:00,  1.19it/s]


Epoch 2, Train Loss: 1.6515, Val Loss: 1.4145
✔ Model improved. Saved to best_cnn_classification.pth


Epoch 3/5: 100%|██████████| 469/469 [06:34<00:00,  1.19it/s]


Epoch 3, Train Loss: 1.5663, Val Loss: 1.3325
✔ Model improved. Saved to best_cnn_classification.pth


Epoch 4/5: 100%|██████████| 469/469 [06:33<00:00,  1.19it/s]


Epoch 4, Train Loss: 1.4995, Val Loss: 1.2733
✔ Model improved. Saved to best_cnn_classification.pth


Epoch 5/5: 100%|██████████| 469/469 [06:33<00:00,  1.19it/s]


Epoch 5, Train Loss: 1.4431, Val Loss: 1.2279
✔ Model improved. Saved to best_cnn_classification.pth
✅ Best model loaded from best_cnn_classification.pth
Test Loss: 1.2093, Test Accuracy: 0.5877


In [34]:
from google.colab import files
files.download("best_cnn_classification.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>